 - [ ] try bigram
 - [x] use pretraining embeddings

In [3]:
import pandas as pd
import os
import numpy as np

In [4]:
# with open(os.path.join(input_path, 'dev_text.txt'), 'r', encoding='utf-8') as f:
#     dev_text = f.read().strip().split('\n')

# with open(os.path.join(input_path, 'heldout_text.txt'), 'r', encoding='utf-8') as f:
#     heldout_text = f.read().strip().split('\n')

In [5]:
# dev_label_path = os.path.join(input_path,'dev_label.txt')
# with open(dev_label_path, 'r', encoding='utf-8') as f:
#     dev_y = f.read().split('\n')

# Prepare data

In [6]:
# dev_data = pd.DataFrame({'text':dev_text, 'label':dev_y})

# dev_data.to_csv(os.path.join(input_path, 'dev_data.tsv'), sep='\t',index=False)

In [157]:
# test_data = pd.DataFrame({'text':heldout_text})
# test_data.to_csv(os.path.join(input_path, 'test_data.tsv'), sep='\t',index=False)

In [217]:
import paths
import baseline_lstm_model
import config
import util
import importlib

In [218]:
reload_packages = [paths, baseline_lstm_model, util, config]
for package in reload_packages:
    importlib.reload(package)
# importlib.reload(data)

# Load data

In [213]:
import torch
from torchtext import data
import random
from torchtext.data import TabularDataset
import torch.nn as nn
import torch.optim as optim



SEED = 1234

torch.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [214]:
dev_datafields = [("text", TEXT), ("label", LABEL)]
dev_dataset = TabularDataset(
               path=os.path.join(paths.input_path, 'dev_data.tsv'),
               format='tsv',
               skip_header=True,
               fields=dev_datafields)

test_datafields = [("text", TEXT)]
test_dataset = TabularDataset(
           path=os.path.join(paths.input_path, 'test_data.tsv'),
           format='csv',
           skip_header=True,
           fields=test_datafields)

In [215]:
train_dataset, valid_dataset = dev_dataset.split(split_ratio=0.7, random_state = random.seed(SEED))

In [216]:
TEXT.build_vocab(train_dataset, 
                 max_size = 40000, 
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_
                )
LABEL.build_vocab(train_dataset)

.vector_cache/glove.6B.zip: 862MB [03:51, 3.73MB/s]                           
100%|█████████▉| 399567/400000 [00:30<00:00, 21531.04it/s]

In [219]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_dataset, valid_dataset), 
    batch_size = config.batch_size,
    device = device,
    sort = True,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True) #sort by length for padding

test_iterator = data.Iterator(
    test_dataset,
    batch_size = config.batch_size, 
    device = device, 
    sort = False, 
    sort_key = lambda x: len(x.text),
    sort_within_batch = True, 
    repeat = False)

In [220]:
train_dataset[0].text[:5]

['A', 'very', 'well', 'made', 'film']

In [221]:
TEXT.vocab.freqs.most_common(1)

[('the', 15700)]

In [222]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f0836fda1e0>, {'neg': 0, 'pos': 1})


# Define model

In [226]:
model = baseline_lstm_model.BaselineLstm(vocab_size=len(TEXT.vocab), 
                                            embed_size=100, 
                                            hidden_size=128, 
                                            output_dim=1,
                                            nlayers=3,
                                            bidirectional=False,
                                            lstm_dropout=0,
                                            dropout=0.5,
                                            pad_idx=TEXT.vocab.stoi[TEXT.pad_token])

In [227]:
model

BaselineLstm(
  (embedding): Embedding(27331, 100, padding_idx=1)
  (lstm): LSTM(100, 128, num_layers=3)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5)
)

In [230]:
pretrained_embeddings = TEXT.vocab.vectors

# load embedding
model.embedding.weight.data.copy_(pretrained_embeddings)
# init unk token
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(100)
model.embedding.weight.data[TEXT.vocab.stoi[TEXT.pad_token]] = torch.zeros(100)

In [231]:
optimizer = optim.Adam(model.parameters(), lr=1e-03, weight_decay=1e-3)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=5e-4)
criterion = nn.BCEWithLogitsLoss()

best_epoch, best_vali_loss, starting_epoch = 0, 400, 0

In [232]:
baseline_lstm_model.run(model, 
                        optimizer, 
                        criterion, 
                        train_iterator, 
                        valid_iterator, 
                        best_epoch=best_epoch, 
                        best_vali_loss=best_vali_loss, 
                        DEVICE=device, 
                        start_epoch=starting_epoch)

### Epoch     0 
Train Loss: 0.6922	Train Acc: 0.5047	Val Loss: 0.6936	Val Acc: 0.4879 
Epoch time used:  5.585192918777466 s 
### Epoch     1 
Train Loss: 0.6918	Train Acc: 0.5047	Val Loss: 0.6936	Val Acc: 0.4879 
Epoch time used:  5.419354200363159 s 
### Epoch     2 
Train Loss: 0.6909	Train Acc: 0.5069	Val Loss: 0.6932	Val Acc: 0.4879 
Epoch time used:  5.4078826904296875 s 
### Epoch     3 
Train Loss: 0.6895	Train Acc: 0.5147	Val Loss: 0.6926	Val Acc: 0.4912 
Epoch time used:  5.408124685287476 s 
### Epoch     4 
Train Loss: 0.6820	Train Acc: 0.6146	Val Loss: 0.6876	Val Acc: 0.5620 
Epoch time used:  5.412967205047607 s 
### Epoch     5 
Train Loss: 0.6771	Train Acc: 0.6165	Val Loss: 0.6860	Val Acc: 0.5587 
Epoch time used:  5.226222991943359 s 
### Epoch     6 
Train Loss: 0.6522	Train Acc: 0.6070	Val Loss: 0.6699	Val Acc: 0.5817 
Epoch time used:  5.217249393463135 s 
### Epoch     7 
Train Loss: 0.6363	Train Acc: 0.6219	Val Loss: 0.6624	Val Acc: 0.5954 
Epoch time used:  5.42